## Setup

## Run this if running on colab

In [8]:
from IPython.display import clear_output; token = input(); clear_output()

In [9]:
! git clone https://$token@github.com/SzymonLukasik/Deep4Life.git

fatal: destination path 'Deep4Life' already exists and is not an empty directory.


In [10]:
%cd /content/Deep4Life

/content/Deep4Life


In [11]:
!pip install anndata

ERROR: Operation cancelled by user


In [ ]:
! pip install pyometiff

## Imports

In [12]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import pyometiff
import os
import gdown
import json
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.model_selection import GridSearchCV

from typing import List
from src.datasets import load_d4ls
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
!gdown 1-0YOHE1VoTRWqfBJLHQorGcHmkhCYvqW

In [7]:
load_d4ls.DATA_PATH

PosixPath('/content/Deep4Life/data')

In [ ]:
!unzip train.zip -d $load_d4ls.DATA_PATH

## Load anndata

In [13]:
train_anndata = load_d4ls.load_full_anndata()

# SVM Baseline

### Dataset Prep

In [2]:
np.random.seed(42)

In [3]:
def get_edge_index(pos, sample_ids, distance_thres):
    # construct edge indexes when there is region information
    edge_list = []
    sample_ids_unique = np.unique(sample_ids)
    for sample_id in sample_ids_unique:
        locs = np.where(sample_ids == sample_id)[0]
        pos_region = pos[locs, :]
        dists = pairwise_distances(pos_region)
        dists_mask = dists < distance_thres
        np.fill_diagonal(dists_mask, 0)
        region_edge_list = np.transpose(np.nonzero(dists_mask)).tolist()
        for i, j in region_edge_list:
            edge_list.append([locs[i], locs[j]])
    return edge_list

In [4]:
def get_train_test_masks(train_anndata, test_count=0):
    sample_ids = train_anndata.obs["sample_id"]
    sample_ids_unique = np.unique(sample_ids)

    sample_ids_idx = np.random.choice(np.arange(len(sample_ids_unique)), test_count, replace=False)
    test_sample_ids_mask = np.zeros_like(sample_ids_unique, dtype=bool)
    test_sample_ids_mask[sample_ids_idx] = True

    test_unique_sample_ids = sample_ids_unique[test_sample_ids_mask]

    test_mask = sample_ids.isin(test_unique_sample_ids)
    train_mask = ~test_mask

    return train_mask, test_mask

In [5]:
def prepare_data(train_anndata, make_graph=False, test_samples=10):
    train_mask, test_mask = get_train_test_masks(train_anndata, test_samples)

    X = train_anndata.layers['exprs']
    X_train = X[train_mask]
    X_test = X[test_mask]

    pos = train_anndata.obs[["Pos_X", "Pos_Y"]].values
    pos_train = pos[train_mask]
    pos_test = pos[test_mask]

    if make_graph:
        sample_ids = train_anndata.obs["sample_id"]
        test_sample_ids = sample_ids[test_mask]
        train_sample_ids = sample_ids[train_mask]

        edges_train = get_edge_index(pos_train, train_sample_ids, 10)
        edges_test = get_edge_index(pos_test, test_sample_ids, 10)
    else:
        edges_train = None
        edges_test = None

    cell_types = np.sort(list(set(train_anndata.obs["cell_labels"].values))).tolist()
    # we here map class in texts to categorical numbers and also save an inverse_dict to map the numbers back to texts
    cell_type_dict = {}
    inverse_dict = {}
    for i, cell_type in enumerate(cell_types):
        cell_type_dict[cell_type] = i
        inverse_dict[i] = cell_type

    Y_train = train_anndata.obs["cell_labels"].values[train_mask]
    Y_test = train_anndata.obs["cell_labels"].values[test_mask]

    Y_train = np.array([cell_type_dict[x] for x in Y_train])
    Y_test = np.array([cell_type_dict[x] for x in Y_test])

    return X_train, Y_train, edges_train, X_test, Y_test, edges_test, inverse_dict



In [14]:
X_train, Y_train, edges_train, X_test, Y_test, edges_test, inverse_dict = prepare_data(train_anndata)

In [15]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler = MinMaxScaler()
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [19]:
train_anndata.obs.head()

,image,sample_id,ObjectNumber,Pos_X,Pos_Y,area,major_axis_length,minor_axis_length,eccentricity,width_px,height_px,acquisition_id,SlideId,Study,Box.Description,Position,SampleId,Indication,BatchId,SubBatchId,ROI,ROIonSlide,includeImage,flag_no_cells,flag_no_ROI,flag_total_area,flag_percent_covered,small_cell,celltypes,flag_tumor,PD1_pos,Ki67_pos,cleavedPARP_pos,GrzB_pos,tumor_patches,distToCells,CD20_patches,Batch,cell_labels
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_1,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,1.0,300.846154,0.692308,13.0,6.094800,2.780135,0.889904,600.0,600.0,2.0,10032145-THOR-VAR-TIS-01-IMC-01,180305_THOR,Slide_IMC-TIS-01,1,10032145-THOR-VAR-TIS-01-PB,THOR,Batch20191023,Batch20191023_01,2,2,1,0,0,0,0,0,MacCD163,0,0,0,0,0,1,8.773580,,Batch20191023,MacCD163
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_3,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,3.0,26.982143,0.928571,56.0,21.520654,3.368407,0.987675,600.0,600.0,2.0,10032145-THOR-VAR-TIS-01-IMC-01,180305_THOR,Slide_IMC-TIS-01,1,10032145-THOR-VAR-TIS-01-PB,THOR,Batch20191023,Batch20191023_01,2,2,1,0,0,0,0,0,Mural,0,0,0,0,0,0,72.247393,,Batch20191023,Mural
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_5,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,5.0,309.083333,0.750000,12.0,5.294329,2.862220,0.841267,600.0,600.0,2.0,10032145-THOR-VAR-TIS-01-IMC-01,180305_THOR,Slide_IMC-TIS-01,1,10032145-THOR-VAR-TIS-01-PB,THOR,Batch20191023,Batch20191023_01,2,2,1,0,0,0,0,0,DC,0,0,0,0,0,1,16.982199,,Batch20191023,DC
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_7,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,7.0,431.916667,0.750000,12.0,5.294329,2.862220,0.841267,600.0,600.0,2.0,10032145-THOR-VAR-TIS-01-IMC-01,180305_THOR,Slide_IMC-TIS-01,1,10032145-THOR-VAR-TIS-01-PB,THOR,Batch20191023,Batch20191023_01,2,2,1,0,0,0,0,0,Tumor,0,0,0,0,0,1,-8.314676,,Batch20191023,Tumor
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_8,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,8.0,116.931034,1.206897,29.0,9.216670,4.112503,0.894932,600.0,600.0,2.0,10032145-THOR-VAR-TIS-01-IMC-01,180305_THOR,Slide_IMC-TIS-01,1,10032145-THOR-VAR-TIS-01-PB,THOR,Batch20191023,Batch20191023_01,2,2,1,0,0,0,0,0,Tumor,0,0,0,0,0,1,-15.358007,,Batch20191023,Tumor


In [20]:
train_anndata.var

,channel,use_channel,marker
0,Y89,1,MPO
1,In113,0,HistoneH3
2,In115,1,SMA
3,Pr141,1,CD16
4,Nd142,1,CD38
5,Nd143,1,HLADR
6,Nd144,1,CD27
7,Nd145,1,CD15
8,Nd146,1,CD45RA
9,Sm147,1,CD163


In [21]:
X_train[:1, :]

array([[0.        , 1.81998789, 0.50264976, 1.2654101 , 0.51500715,
        3.33126057, 0.40379979, 0.45542107, 0.52873423, 2.94839226,
        2.27798847, 0.        , 3.99579148, 0.86617603, 0.18491241,
        0.52133242, 1.66338067, 0.49026304, 0.49894161, 0.41989374,
        0.23960538, 0.25355285, 0.77554996, 1.48411277, 0.35955019,
        0.08998893, 0.33325552, 0.75729804, 0.97357613, 0.03106147,
        0.79221775, 0.56723839, 1.6516099 , 2.55621816, 0.77737784,
        0.        , 3.58108228, 0.42427316, 4.76113044, 5.30135121]])

In [22]:
X_train_scaled[:1, :]

array([[0.        , 0.24702849, 0.10195528, 0.26191336, 0.1014993 ,
        0.48290376, 0.10005249, 0.06144762, 0.11736924, 0.55040873,
        0.36014022, 0.        , 0.61227858, 0.13580361, 0.02568145,
        0.10570444, 0.34137134, 0.12064267, 0.10746827, 0.07982855,
        0.03751061, 0.04253901, 0.15124285, 0.17462497, 0.05631266,
        0.01777021, 0.05738733, 0.10912608, 0.17529799, 0.00446232,
        0.15982717, 0.096778  , 0.24820902, 0.42255869, 0.11643712,
        0.        , 0.58804124, 0.0798684 , 0.6430607 , 0.66239967]])

In [23]:
X_train.shape

(219764, 40)

(219764, 40)

## Grid Search on sampled dataset

In [15]:
n_samples = 10_000

sample_perm = np.random.permutation(np.arange(X_train.shape[0]))[:n_samples]
X_train_sampled = X_train[sample_perm]
Y_train_sampled = Y_train[sample_perm]

In [ ]:
LinearSVC_param_grid = {
    'penalty': ['l1', 'l2'],
    'dual': [False],
    'C': [0.1, 0.3, 0.5, 0.7, 0.9],
}

linear_svc_grid_search = GridSearchCV(LinearSVC(), LinearSVC_param_grid, verbose=1, n_jobs=-1)
linear_svc_grid_search.fit(X_train_sampled, Y_train_sampled)

In [ ]:
pd.DataFrame(linear_svc_grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_dual,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,22.375490,1.407985,0.002695,0.000459,0.1,False,l1,"{'C': 0.1, 'dual': False, 'penalty': 'l1'}",0.9210,0.9195,0.9210,0.9200,0.9275,0.9218,0.002909,9
1,0.819687,0.167843,0.002435,0.000134,0.1,False,l2,"{'C': 0.1, 'dual': False, 'penalty': 'l2'}",0.9150,0.9190,0.9225,0.9165,0.9280,0.9202,0.004654,10
2,32.245410,1.343506,0.002789,0.000190,0.3,False,l1,"{'C': 0.3, 'dual': False, 'penalty': 'l1'}",0.9205,0.9195,0.9265,0.9195,0.9295,0.9231,0.004128,5
3,0.761785,0.025140,0.002304,0.000052,0.3,False,l2,"{'C': 0.3, 'dual': False, 'penalty': 'l2'}",0.9170,0.9200,0.9245,0.9185,0.9330,0.9226,0.005774,8
4,34.447949,1.629052,0.002839,0.000412,0.5,False,l1,"{'C': 0.5, 'dual': False, 'penalty': 'l1'}",0.9195,0.9205,0.9275,0.9200,0.9335,0.9242,0.005492,2
5,1.053312,0.354597,0.002532,0.000352,0.5,False,l2,"{'C': 0.5, 'dual': False, 'penalty': 'l2'}",0.9185,0.9205,0.9250,0.9180,0.9335,0.9231,0.005757,4
6,35.933432,1.767400,0.004755,0.003147,0.7,False,l1,"{'C': 0.7, 'dual': False, 'penalty': 'l1'}",0.9195,0.9205,0.9265,0.9205,0.9335,0.9241,0.005314,3
7,0.823248,0.020690,0.002277,0.000023,0.7,False,l2,"{'C': 0.7, 'dual': False, 'penalty': 'l2'}",0.9190,0.9200,0.9255,0.9190,0.9315,0.9230,0.004889,6
8,35.097073,3.953730,0.004401,0.002097,0.9,False,l1,"{'C': 0.9, 'dual': False, 'penalty': 'l1'}",0.9190,0.9215,0.9280,0.9200,0.9335,0.9244,0.005526,1
9,1.044367,0.288786,0.003142,0.001208,0.9,False,l2,"{'C': 0.9, 'dual': False, 'penalty': 'l2'}",0.9180,0.9205,0.9250,0.9180,0.9325,0.9228,0.005483,7


In [ ]:
SVC_param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf']
    }


svc_grid_search = GridSearchCV(SVC(), SVC_param_grid, refit = True, verbose = 1)
svc_grid_search.fit(X_train_sampled, Y_train_sampled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf']},
             verbose=1)

In [ ]:
pd.DataFrame(svc_grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.500914,0.510002,1.574526,1.491782,0.1,scale,rbf,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}",0.8815,0.8940,0.8870,0.8825,0.8840,0.8858,0.004501,8
1,2.593324,0.851279,0.881963,0.176090,0.1,auto,rbf,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}",0.8845,0.8955,0.8895,0.8825,0.8870,0.8878,0.004512,7
2,1.172456,0.221427,0.616334,0.130661,1,scale,rbf,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.9300,0.9425,0.9325,0.9310,0.9330,0.9338,0.004479,6
3,1.079113,0.021274,0.553207,0.011781,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.9370,0.9415,0.9365,0.9330,0.9345,0.9365,0.002881,5
4,1.030831,0.233684,0.534230,0.112337,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.9460,0.9475,0.9460,0.9395,0.9415,0.9441,0.003056,1
5,1.168107,0.264840,0.571890,0.119006,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.9385,0.9450,0.9465,0.9365,0.9405,0.9414,0.003800,2
6,0.962171,0.039626,0.452298,0.008006,100,scale,rbf,"{'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}",0.9395,0.9445,0.9405,0.9335,0.9340,0.9384,0.004152,4
7,1.210361,0.241654,0.566815,0.122539,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.9370,0.9450,0.9425,0.9335,0.9380,0.9392,0.004082,3


In [16]:
svc_best_configuration = {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
svc = SVC(**svc_best_configuration)
svc.fit(X_train_scaled, Y_train)

SVC(C=10, gamma='auto')

In [ ]:
svc.score(X_train_scaled, Y_train)

In [17]:
svc.score(X_test_scaled, Y_test)

0.8890585540611969

In [ ]:
# fit an svm model

SVC_param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['linear']}
# C is the penalty parameter of the error term. It controls the trade off between smooth decision boundary and classifying the training points correctly.
# gamma is the parameter of the RBF kernel and can be thought of as the ‘spread’ of the kernel and therefore the decision region.
# kernel is the type of kernel used in the algorithm. The most common ones are ‘linear’, ‘poly’, and ‘rbf’.

SVC_grid = GridSearchCV(SVC(), SVC_param_grid, refit = True, verbose = 1)
SVC_grid.fit(X_train_scaled[:trim], Y_train[:trim])

In [ ]:
trim = 50_000
svc = SVC(C=1, gamma=0.1, kernel='linear', verbose=True)
svc.fit(X_train_scaled[:trim], Y_train[:trim])

[LibSVM]

SVC(C=1, gamma=0.1, kernel='linear', verbose=True)

In [ ]:
svc.score(X_train_scaled[trim:], Y_train[trim:])

0.8770681124125401

In [ ]:
svc.score(X_train_scaled, Y_train)

0.8813055422397581

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [ ]:
svc.score(X_test_scaled, Y_test)

0.8108271586269469

## Fit on the full dataset

In [ ]:
trim = len(X_train)
svc_larger = SVC(C=1, gamma=0.1, kernel='linear', verbose=True)
svc_larger.fit(X_train_scaled[:trim], Y_train[:trim])

[LibSVM]

SVC(C=1, gamma=0.1, kernel='linear', verbose=True)

In [ ]:
svc_larger.score(X_train_scaled[trim:], Y_train[trim:])

0.9116153145814366

In [ ]:
svc_larger.score(X_train_scaled, Y_train)

0.8813055422397581

In [ ]:
svc_larger.score(X_test, Y_test)

0.4135289077070417

In [ ]:
svc_larger.score(X_test_scaled, Y_test)

0.8718685417611775